In [6]:
import pickle
import sys
sys.path.append('../')

from pipeline_src.dataset.prompt_schemas import (
    hypo_term_hyper,
    predict_child_from_2_parents,
    predict_child_from_parent,
    predict_child_with_parent_and_grandparent,
    predict_children_with_parent_and_brothers,
    predict_parent_from_child_granparent,
    predict_parent_from_child,
    predict_multiple_parents_from_child,
)

In [7]:
with open('./custom_datasets/1B.italian_train.pickle', 'rb') as f:
    train = pickle.load(f)

In [9]:
transforms={
        "only_child_leaf": predict_child_with_parent_and_grandparent,  # заменить на предсказание ребенка
        "only_leafs_all": predict_child_from_parent,
        "only_leafs_divided": predict_children_with_parent_and_brothers,
        "leafs_and_no_leafs": predict_child_from_parent,
        "simple_triplet_grandparent": predict_parent_from_child_granparent,
        "simple_triplet_2parent": predict_child_from_2_parents,
        "predict_hypernym": predict_parent_from_child,
        "predict_multiple_hypernyms": predict_multiple_parents_from_child,
    }



In [10]:

elem = train[0]
case = elem['case']
processed_term, target = transforms[case](elem)

In [12]:
processed_term + target

'hyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,'

In [31]:
# 1 - 2 - 5 - 15 - 30 - 50
options = [1, 2, 5, 15, 30, 50]

for num_few_shot in options:

    resulting_prompt = ''

    for i in range(num_few_shot):
        resulting_prompt += '\n'
        elem = train[i]
        case = elem['case']
        processed_term, target = transforms[case](elem)
        resulting_prompt += processed_term + target

    print(resulting_prompt)
    with open(f'./few_shots/italian_{num_few_shot}shot.txt', 'w') as f:
        f.write(resulting_prompt)


hyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,

hyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,
hyponym: Sigillo | hypernyms:comune, municipalità, comune italiano, frazione, paese, quartiere, comunità, borgo,

hyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,
hyponym: Sigillo | hypernyms:comune, municipalità, comune italiano, frazione, paese, quartiere, comunità, borgo,
hyponym: toga | hypernyms:comune, mantella, indumento, vestito, mantello, veste, abito, costume,
hyponym: Eduard Buchner | hypernyms:accademico, persona, professore, biochimico, scienziato, chimico, scopritore, docente,
hyponym: montagna | hypernyms:panorama, grande quantità, rilievo,

hyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,
hyponym: Sigillo | hypernyms:comune, municipalità, comune italiano, frazione, paese, quartiere, comunità, borgo,
hyponym: toga | hypernyms:comune, mantella, indumento, vestito, mantello, veste, abito, costume


hyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,


In [18]:
from pipeline_src.dataset.dataset import HypernymDataset
from transformers import AutoTokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [20]:
with open('./few_shots/italian_5shot.txt', 'r') as f:
    few_shot_text = f.readlines()


In [24]:
''.join(few_shot_text)

'\nhyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,\nhyponym: Sigillo | hypernyms:comune, municipalità, comune italiano, frazione, paese, quartiere, comunità, borgo,\nhyponym: toga | hypernyms:comune, mantella, indumento, vestito, mantello, veste, abito, costume,\nhyponym: Eduard Buchner | hypernyms:accademico, persona, professore, biochimico, scienziato, chimico, scopritore, docente,\nhyponym: montagna | hypernyms:panorama, grande quantità, rilievo,'

In [25]:
dataset = HypernymDataset(
    './custom_datasets/1B.italian.pickle',
    tokenizer,
    few_shot_text=''.join(few_shot_text)
)

In [27]:
batch = dataset[0]

In [28]:
tokenizer.decode(batch['encoded_term'])

'<s>[INST] <<SYS>> You are a helpfull assistant. List all the possible words divided with a coma. Your answer should not include anything except the words divided by a coma<</SYS>>\nhyponym: sesto | hypernyms:grado, numero ordinale, frazione, carica,\nhyponym: Sigillo | hypernyms:comune, municipalità, comune italiano, frazione, paese, quartiere, comunità, borgo,\nhyponym: toga | hypernyms:comune, mantella, indumento, vestito, mantello, veste, abito, costume,\nhyponym: Eduard Buchner | hypernyms:accademico, persona, professore, biochimico, scienziato, chimico, scopritore, docente,\nhyponym: montagna | hypernyms:panorama, grande quantità, rilievo,\nhyponym: tecnologia | hypernyms:[/INST]'